In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('C:\\Users\\Predator\\Downloads\\diabetes.csv')

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [14]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000


In [11]:
column_name = 'Outcome'

In [13]:
data[column_name] = data[column_name].astype('object')

In [15]:
data_prep = pd.get_dummies(data, drop_first = True)

In [16]:
data_prep.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome_1
0,6,148,72,35,0,33.6,0.627,50,True
1,1,85,66,29,0,26.6,0.351,31,False
2,8,183,64,0,0,23.3,0.672,32,True
3,1,89,66,23,94,28.1,0.167,21,False
4,0,137,40,35,168,43.1,2.288,33,True


In [19]:
X = data_prep.iloc[:,:-1]
y = data_prep.iloc[:,-1]

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, stratify = y)

In [18]:
from xgboost import XGBClassifier

In [24]:
XGB = XGBClassifier(n_jobs = -1)

In [23]:
from sklearn.model_selection import GridSearchCV

In [25]:
xgb_params = {
    'n_estimators':[10,20,35,50],
    'max_depth':[2,4,6,8]
}

In [28]:
grid = GridSearchCV(estimator = XGB,
                   param_grid = xgb_params,
                   scoring = 'accuracy',
                   n_jobs = -1,
                   cv = 5,
                   return_train_score = True)

In [38]:
grid_fit = grid.fit(X, y)

In [39]:
grid_results_xgb = pd.DataFrame.from_dict(grid_fit.cv_results_)

In [40]:
grid_top_rank = grid_results_xgb[grid_results_xgb['rank_test_score'] == 1]

In [41]:
grid_top_rank

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
4,0.029097,0.00383,0.007408,0.000386,4,10,"{'max_depth': 4, 'n_estimators': 10}",0.727273,0.727273,0.75974,...,0.768347,0.045058,1,0.876221,0.885993,0.869707,0.863415,0.869919,0.873051,0.007634


In [42]:
from sklearn.model_selection import RandomizedSearchCV

In [62]:
rs = RandomizedSearchCV(estimator = XGB,
                       param_distributions = xgb_params,
                       n_iter = 12,
                       cv = 5,
                       scoring = 'accuracy',
                       return_train_score = True,
                       n_jobs = -1)

In [63]:
rs_fit = rs.fit(X,y)

In [64]:
rs_results_xgb = pd.DataFrame.from_dict(rs_results_xgb.cv_results_)

AttributeError: 'DataFrame' object has no attribute 'cv_results_'

In [50]:
rs_top_rank = rs_results_xgb[rs_results_xgb['rank_test_score'] == 1]

In [52]:
pd.options.display.max_columns = 30

In [53]:
rs_top_rank

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
8,0.028673,0.001721,0.008668,0.001154,20,2,"{'n_estimators': 20, 'max_depth': 2}",0.74026,0.818182,0.766234,0.675325,0.727273,0.779221,0.831169,0.818182,...,0.768216,0.045349,1,0.823444,0.816208,0.82055,0.82055,0.821997,0.819103,0.813314,0.82055,0.839595,0.816474,0.821179,0.006776


## Checking quality differences

In [55]:
XGB = XGBClassifier(n_jobs = -1, max_depth = 4, n_estimators = 10).fit(X_train, y_train)

In [56]:
xgb_pred = XGB.predict(X_test)

In [57]:
from sklearn.metrics import classification_report

In [58]:
print(classification_report(y_test, xgb_pred))

              precision    recall  f1-score   support

       False       0.82      0.83      0.83       100
        True       0.68      0.67      0.67        54

    accuracy                           0.77       154
   macro avg       0.75      0.75      0.75       154
weighted avg       0.77      0.77      0.77       154



In [59]:
XGB2 = XGBClassifier(n_jobs = -1, max_depth = 2, n_estimators = 20).fit(X_train, y_train)

In [60]:
xgb2_pred = XGB2.predict(X_test)

In [61]:
print(classification_report(y_test, xgb2_pred))

              precision    recall  f1-score   support

       False       0.79      0.84      0.81       100
        True       0.66      0.57      0.61        54

    accuracy                           0.75       154
   macro avg       0.72      0.71      0.71       154
weighted avg       0.74      0.75      0.74       154

